<a href="https://colab.research.google.com/github/Enamulla577/EVAI/blob/main/Assignment-4/EMNIST_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [20]:
#Loading EMNIST dataset
train_set = torchvision.datasets.EMNIST(
    root='./data',
    train = True,
    split = 'bymerge',
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)


In [66]:
train_loader = torch.utils.data.DataLoader(train_set
    ,batch_size=100
    ,shuffle=True
)

In [60]:
# Define the Model

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1) # input 28x28x1 | (3x3x1)x10  output-28x28x10 RF=3
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, padding=1) # input 28x28x1 | (3x3x10)x10  28X28X10 RF=5
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, padding=1) #after max pooling 14x14x10 |(3x3x10)x20  14x14x20 RF=12
        self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3, padding=1) #input 14x14x20|(3x3x20)x20   14x14x20 RF=14
        self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3) #after max pooling 7x7x20 | 3x3x20x30  5x5x30 RF=30
        self.conv6 = nn.Conv2d(in_channels=30, out_channels=10, kernel_size=3) #5x5x30 | (3x3x30)10 3x3x10 RF=32
        self.out = nn.AvgPool2d(3, stride=2)  #1x1x10 RF=34
        

    def forward(self, t):
        #  input layer
        t = t

        # (1) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        
        # (2) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
       
        # (3) hidden conv layer
        t = self.conv3(t)
        t = F.relu(t)
        
        # (4) hidden conv layer
        t = self.conv4(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (5) hidden conv layer
        t = self.conv5(t)
        t = F.relu(t)

        # (6) hidden conv layer
        t = self.conv6(t)
        t = F.relu(t)
        # output layer
        t = self.out(t)
        # t = t.softmax(t, dim=1)
      
        return t

In [61]:
network = Network()
print(network)

Network(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(30, 10, kernel_size=(3, 3), stride=(1, 1))
  (out): AvgPool2d(kernel_size=3, stride=2, padding=0)
)


In [62]:
sample = next(iter(train_set)) 
image, label = sample
pred = network(image.unsqueeze(0))
pred

tensor([[[[0.0000e+00]],

         [[0.0000e+00]],

         [[7.4309e-02]],

         [[0.0000e+00]],

         [[7.8198e-05]],

         [[4.7964e-03]],

         [[0.0000e+00]],

         [[4.0292e-02]],

         [[2.5999e-02]],

         [[1.2490e-02]]]], grad_fn=<AvgPool2DBackward>)

In [63]:
from torchsummary import summary


model = Network()  # copying the net to the device
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
            Conv2d-2           [-1, 10, 28, 28]             910
            Conv2d-3           [-1, 20, 14, 14]           1,820
            Conv2d-4           [-1, 20, 14, 14]           3,620
            Conv2d-5             [-1, 30, 5, 5]           5,430
            Conv2d-6             [-1, 10, 3, 3]           2,710
         AvgPool2d-7             [-1, 10, 1, 1]               0
Total params: 14,590
Trainable params: 14,590
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.19
Params size (MB): 0.06
Estimated Total Size (MB): 0.24
----------------------------------------------------------------


In [64]:
network = Network()
optimizer = optim.Adam(network.parameters(), lr=0.01)
train_loader = torch.utils.data.DataLoader(
    train_set
    ,batch_size=100
    ,shuffle=True
)

In [65]:
for epoch in range(20):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

RuntimeError: ignored